<a href="https://colab.research.google.com/github/markmaloba/DSCORE-Week-9-Independent-Project/blob/master/(KNN)_DSCORE_Week_9_Independent_Project_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

You just woke up. You think you heard your phone ringing, but you're just imagining things. The room looks kind of unfamiliar. All you remember is being out last night. "Whose apartment is this? What's with the overdone rustic look. They've kind of taken it too far. Did I go home with someone?" you ask, racking your brain super hard. Hmm... &#x1F914; You don't remember meeting anyone you were particularly attracted to. It's all a blur, except for that very old, lanky white man with the long beard and pointy hat. He used a staff just as tall as him to support his old frame. And why the heck was he in a robe. Meh, you brush the thought away and scan the room. Your eyes catch what looks like a ticket on a bedside stand. Apart from those items, yourself, and the bed there's nothing more in the room. 

Studying the ticket a second time, a few things stand out: "RMS Titanic". That's odd, is it a movie ticket? You remember your partner bawling their eyes out watching *Titanic* a few movie nights ago. You focus back on the ticket. Boarding pass, yadda yadda third class yadda yadda, "Date: 10th April 1912". Wait! Nooooo! It can't be!....



---
.

.

The RMS Titanic was a British passenger liner operated by the White Star Line that sank in the North Atlantic Ocean in the early morning hours of April 15, 1912, after striking an iceberg during her maiden voyage from Southampton to New York City. Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters.

### Defining the question

Using the provided data, Can we design a model that correctly categorizes passengers aboard the *Titanic* as either survivors, or deceased? 

### Metric for success

Heavy is the head that wears the crown. A gargantuan task lies before you. After reading the background section, surely the objective is clear. You want the model to tell which passengers have lowest chances of surviving so that you know where to focus your efforts when you time-travel back to 1912. **THESE ARE HUMAN LIVES**, so we want 100% accuracy. That's the bar. No room for error!

More realistically, we can shoot for **80 percent**

### Experimental design choices

We will use a the K Nearest neighbors classifier for this model, with varying test and train sizes and K values, to see how these changes affect our accuracy.

### Appropriateness of the data to answer our question

The total number of passengers on the ill-fated Titanic was 2,224. Of those, we had 1317 passengers. Our data comprises 891 records. It is important to note that our dataset does not capture accomodation and boarding details for any of the staff, so we can't predict their survival rates. The missing 60 percent of the data may drastically improve our model if added to our data.

However, we do have 67 percent of passenger data which is a pretty darn good sample size as far as predicting PASSENGER survival goes.

In [0]:
#Importing libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
#load test dataset

testdf = pd.read_csv('test_1.csv')

In [35]:
#preview test dataset

testdf.head(15)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [36]:
# Checking shape of the dataset

testdf.shape


(418, 11)

In [0]:
#load 2nd dataset

traindf = pd.read_csv('train_5.csv')

In [38]:
# preview training set

traindf.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Understanding our data

In [39]:
testdf.shape

(418, 11)

In [40]:
traindf.shape

(891, 12)

In [41]:
testdf.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [42]:
traindf.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

All the data types used are appropriate.

The test dataframe has one less column than the train set. That is the survived column.

We can use this as our dependent variable. We can test all the other features and have the model predict whether a passenger survived or not.



# Univariate & Bivariate analysis

In [43]:
import pandas_profiling

pandas_profiling.ProfileReport(traindf)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,12
Number of observations,891
Total Missing (%),8.1%
Total size in memory,83.7 KiB
Average record size in memory,96.1 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [44]:
# Checking dimensionality of the columns. I especially want to see number of unique items as far as cabins go.

traindf['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

I feel that the ticket number column does not have any bearing the survival rates for the passengers so I'll drop them in the next cell.

I will also, RELUCTANTLY drop the cabin column since it's a little tiresome to encode. The format for the cabins isn't standard. If my accuracy is low, I might reconsider working with that column.

I'm happy that the **fare** serves as a proxy for the access a passenger had to a cabin, and cabin type, if they indeed had a cabin. First class accomodation will cost more than 3rd class and so on.

Also, the way the ship was structured, people in a certain class of accomodation were all situated in a certain part of the ship.

Considering that chaos on the Titanic (hitting the iceberg and the mess that ensued) started at 2340H, I am of the opinion that this accomodation data is super important since everyone was starting from a different point, as they rushed to get to the lifeboats, and that directly affects the probability that they survive.

# Modeling (K-Nearest Neighbors)

In [45]:
traindf.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [46]:
traindf2.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [0]:
# Rearranging the columns, to have our predicted column last

traindf = traindf[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived']]


There's other ways to set x and y, but I chose to do this just because it makes good practice as far as column relocation is concerned.

In [48]:
# Confirmation

traindf.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [0]:
# Dropping the columns not needed for our analysis, as stated before

traindf2 = traindf.drop(['PassengerId', 'Name', 'Ticket','Cabin','Embarked'], axis=1)

In [50]:
# COnfirmation of dropped columns

traindf2.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,22.0,1,0,7.2500,0
1,1,female,38.0,1,0,71.2833,1
2,3,female,26.0,0,0,7.9250,1
3,1,female,35.0,1,0,53.1000,1
4,3,male,35.0,0,0,8.0500,0


We need to encode the sex column before we can run our algorithm

In [51]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder

label_encoder = preprocessing.LabelEncoder() 

# Encoding the sex column

traindf2['Sex']= label_encoder.fit_transform(traindf2['Sex']) 
  
traindf2['Sex'].unique()

array([1, 0])

In [52]:
#Encoding confirmation

traindf2.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,1,22.0,1,0,7.2500,0
1,1,0,38.0,1,0,71.2833,1
2,3,0,26.0,0,0,7.9250,1
3,1,0,35.0,1,0,53.1000,1
4,3,1,35.0,0,0,8.0500,0


Note: Male = 1, and Female = 2 for the sex column

In [53]:
# Splitting our dataset into its attributes and labels
# ---
# The X variable contains the first four columns of the dataset (i.e. attributes) while y contains the labels.
# ---
# 
x = traindf2.iloc[:, :-1].values
# x = traindf2 [['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
#        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
#print(X)
y = traindf2.iloc[:, 6].values
print(y)

[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0
 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1
 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0
 0 0 1 1 0 1 0 0 1 0 0 0 

In [0]:
# Train and Test set Split
# ---

# ---
# 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
# Feature Scaling to ensure uniformity

# 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [0]:
# Training and Predictions
# ---
# The first step is to import the KNeighborsClassifier class from the sklearn.neighbors library. 
# In the second line, this class is initialized with one parameter, i.e. n_neigbours. 
# This is basically the value for the K. There is no ideal value for K and it is selected after testing and evaluation, 
# however to start out, 5 seems to be the most commonly used value for KNN algorithm.
# ---
# 
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

ValueError: ignored

Let's identify where the NaN's are and fix that problem.

In [0]:
traindf2.isnull().any()

Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Survived    False
dtype: bool

The age column is the problematic one. Let's see the null count in that column.

In [0]:
traindf2['Age'].isnull().sum(axis = 0)

177

I am going to try first dropping the null values in the age column. After assessing the model's accuracy, I'll see if imputing affects the accuracy positively or negatively.

In [0]:
# Backup 

traindf3 = traindf2

In [0]:
traindf2 = traindf2.dropna(axis=0, subset=['Age'])

In [0]:
# Confirmation

traindf2['Age'].isnull().sum(axis = 0)

0

Now we were successful. Let's continue.

In [0]:
# Resuming the training and predictions

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(x_test)

And now we will generate the confusion matrix and classification report, to evaluate our model

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print ('==CONFUSION MATRIX==')
print(confusion_matrix(y_test, y_pred))
print ('---')
print ('==CLASSIFICATION REPORT==')
print(classification_report(y_test, y_pred))

==CONFUSION MATRIX==
[[67 17]
 [14 45]]
---
==CLASSIFICATION REPORT==
              precision    recall  f1-score   support

           0       0.83      0.80      0.81        84
           1       0.73      0.76      0.74        59

    accuracy                           0.78       143
   macro avg       0.78      0.78      0.78       143
weighted avg       0.79      0.78      0.78       143



# Hyperparameter Tuning

#### First: effect when we impute missing ages

In [0]:
traindf4 = traindf3

In [0]:
x = traindf4.iloc[:, :-1].values
y = traindf4.iloc[:, 6].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [0]:
from sklearn.impute import SimpleImputer

age = ['Age']

imputer = SimpleImputer(strategy = 'median')

traindf4[age] = imputer.fit_transform(traindf4[age])

In [27]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

ValueError: ignored

In [30]:
traindf4.isnull().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
dtype: bool

Even after imputing and confirming there are no null values, we still get the error above. I will try to encode any infinity values with "0".

In [31]:
np.nan_to_num(traindf4)

array([[ 0.    ,  3.    ,  1.    , ...,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    ,  0.    , ...,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    ,  0.    , ...,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 0.    ,  3.    ,  0.    , ...,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  1.    ,  1.    , ...,  0.    ,  0.    , 30.    ],
       [ 0.    ,  3.    ,  1.    , ...,  0.    ,  0.    ,  7.75  ]])

In [32]:
# 2nd attempt

# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors=5)
# classifier.fit(x_train, y_train)

ValueError: ignored

I'm going to leave this thread as is, since I can't find another solution.

Any help from anyone reading this will be greatly appreciated.

In the next sections, I will keep using the dataset with the null values in the Age column dropped.

In [28]:
# y_pred = classifier.predict(x_test)

ValueError: ignored

In [0]:
# from sklearn.metrics import classification_report, confusion_matrix
# print ('==CONFUSION MATRIX==')
# print(confusion_matrix(y_test, y_pred))
# print ('---')
# print ('==CLASSIFICATION REPORT==')
# print(classification_report(y_test, y_pred))

### Second: effect when we increase test subset to 30% (n = 5 | 70/30 split)

In [0]:
traindf5 = traindf2

In [0]:
x = traindf5.iloc[:, :-1].values
y = traindf5.iloc[:, 6].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [65]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(x_test)

In [67]:
from sklearn.metrics import classification_report, confusion_matrix
print ('==CONFUSION MATRIX==')
print(confusion_matrix(y_test, y_pred))
print ('---')
print ('==CLASSIFICATION REPORT==')
print(classification_report(y_test, y_pred))

==CONFUSION MATRIX==
[[108  15]
 [ 31  61]]
---
==CLASSIFICATION REPORT==
              precision    recall  f1-score   support

           0       0.78      0.88      0.82       123
           1       0.80      0.66      0.73        92

    accuracy                           0.79       215
   macro avg       0.79      0.77      0.78       215
weighted avg       0.79      0.79      0.78       215



### Third: effect when we increase test subset to 40% (n = 5 | 60/40 split)

In [0]:
traindf6 = traindf2

In [0]:
x = traindf6.iloc[:, :-1].values
y = traindf6.iloc[:, 6].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.40)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [72]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(x_test)

In [74]:
from sklearn.metrics import classification_report, confusion_matrix
print ('==CONFUSION MATRIX==')
print(confusion_matrix(y_test, y_pred))
print ('---')
print ('==CLASSIFICATION REPORT==')
print(classification_report(y_test, y_pred))

==CONFUSION MATRIX==
[[136  32]
 [ 33  85]]
---
==CLASSIFICATION REPORT==
              precision    recall  f1-score   support

           0       0.80      0.81      0.81       168
           1       0.73      0.72      0.72       118

    accuracy                           0.77       286
   macro avg       0.77      0.76      0.77       286
weighted avg       0.77      0.77      0.77       286



### Fourth: effect when we increase number of neighbors (n = 7 | 80/20 split)

In [0]:
traindf7 = traindf2

In [0]:
x = traindf7.iloc[:, :-1].values
y = traindf7.iloc[:, 6].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [79]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=7)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(x_test)

In [81]:
from sklearn.metrics import classification_report, confusion_matrix
print ('==CONFUSION MATRIX==')
print(confusion_matrix(y_test, y_pred))
print ('---')
print ('==CLASSIFICATION REPORT==')
print(classification_report(y_test, y_pred))

==CONFUSION MATRIX==
[[73 12]
 [19 39]]
---
==CLASSIFICATION REPORT==
              precision    recall  f1-score   support

           0       0.79      0.86      0.82        85
           1       0.76      0.67      0.72        58

    accuracy                           0.78       143
   macro avg       0.78      0.77      0.77       143
weighted avg       0.78      0.78      0.78       143



### Fifth: effect when we decrease number of neighbors (n = 3 | 80/20 split)

In [0]:
traindf8 = traindf2

In [0]:
x = traindf8.iloc[:, :-1].values
y = traindf8.iloc[:, 6].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [86]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(x_test)

In [88]:
from sklearn.metrics import classification_report, confusion_matrix
print ('==CONFUSION MATRIX==')
print(confusion_matrix(y_test, y_pred))
print ('---')
print ('==CLASSIFICATION REPORT==')
print(classification_report(y_test, y_pred))

==CONFUSION MATRIX==
[[77  4]
 [13 49]]
---
==CLASSIFICATION REPORT==
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        81
           1       0.92      0.79      0.85        62

    accuracy                           0.88       143
   macro avg       0.89      0.87      0.88       143
weighted avg       0.89      0.88      0.88       143



# Challenging our solution & Conclusion

Interesting findings!

Increasing the size of the test set improves our accuracy, but after some point the scales tip and the law of diminishing returns takes over.

Increasing the size of 'n' (number of neighbors our model looks at when making comparisons) decreases our accuracy.

However, decreasing the size of 'n' SENDS OUR ACCURACY INTO THE FRICKING STRATOSPHERE!

I was not expecting that much of an improvement! 	&#128563;

I'm proud to say that the new accuracy level stands at EIGHTY EIGHT PERCENT! 

Armed with this model, I valiantly accept my mission to go back to 1912 and stem the loss of lives that occured as a result of the Titanic sinking! &#9876;&#128737; 